# Setup

In [1]:
import torch
import argparse
import numpy as np

parser = argparse.ArgumentParser()

parser.add_argument('--seed', type=int, default=123)
parser.add_argument('--method', type=str, default='CER', choices=[
    'CER', 'REF', 'SLL', 'SLL_G'
])
parser.add_argument('--budget_pct', type=float, default=0.25)
parser.add_argument('--g0_method', type=str, default='random', choices=[
  'random', # randomly distribution of g0
  'bias', # a random class has a 3x higher likelihood of being in g0
  'large_cluster', # a random node and [g0_size] of its neighbors are in g0
  # 'many_clusters', # 10 random nodes and [g0_size] of their neighbors are in g0
  ])
parser.add_argument('--g0_size', type=float, default=0.2)
parser.add_argument('--lr', type=float, default=0.1)
parser.add_argument('--T_s', type=int, default=-1)
parser.add_argument('--T_u', type=int, default=0)
parser.add_argument('--dataset', type=str, default='cora', choices=[
    'Cora', 'Cora-ML', 'Citeseer', 'Pubmed', 'Polblogs', 'ACM', 'BlogCatalog', 'Flickr', 'UAI'
])
parser.add_argument('--ptb_rate', type=float, default=0.25)

# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
args = parser.parse_args("")

device = "cuda:1" if torch.cuda.is_available() else "cpu"
# device = 'cpu'

np.random.seed(args.seed)
torch.manual_seed(args.seed)

In [112]:
import utils
importlib.reload(utils)

adj, feat, labels, train_mask, val_mask, test_mask = utils.load_data(args.dataset, args.seed)

Loading cora dataset...
Selecting 1 largest connected components


In [71]:
from deeprobust.graph.data import Dataset
from deeprobust.graph.utils import preprocess

clean_dataset = Dataset(root='./tmp/', name=args.dataset, seed=args.seed)
adj, feat, labels = clean_dataset.adj, clean_dataset.features, clean_dataset.labels
adj, feat, labels = preprocess(adj, feat, labels, preprocess_adj=False, device='cpu') # conver to tensor
idx_train, idx_val, idx_test = clean_dataset.idx_train, clean_dataset.idx_val, clean_dataset.idx_test
# adj = torch.tensor(clean_dataset.adj.toarray(), dtype=torch.float).to(device)
# feat = torch.tensor(clean_dataset.features.toarray(), dtype=torch.float).to(device)
# label = torch.tensor(clean_dataset.labels, dtype=torch.long).to(device)

train_mask = torch.zeros([adj.shape[0]], dtype=torch.bool)  
train_mask[idx_train] = 1
test_mask = torch.zeros([adj.shape[0]], dtype=torch.bool)  
test_mask[idx_test] = 1
val_mask = torch.zeros([adj.shape[0]], dtype=torch.bool)  
val_mask[idx_val] = 1

num_samples = 2560
subgraph_size = 64

Loading cora dataset...
Selecting 1 largest connected components


In [109]:
import utils
import importlib
importlib.reload(utils)

T_s = T_u = torch.tensor([0])

if args.T_s == -1: T_s = labels
else:
    T_s = utils.binary(feat[:,args.T_s].clone())
    feat[:,args.T_s] = 0

if args.T_u == -1: T_u = labels
else:
    T_u = utils.binary(feat[:,args.T_u].clone())
    feat[:,args.T_u] = 0

# T_s = T_s.to(device)
# T_u = T_u.to(device)
# import torch.nn.functional as F
# fs = F.normalize(feat, p=1, dim=1).sum(axis=1)
# T_u = (fs > fs.median()).long()

In [110]:
g0_size = int(args.g0_size * adj.shape[0])
g0 = utils.get_g0(g0_size, adj, method=args.g0_method)

G0 with method: random
G0 size: 487
G0 pct: 19.60%


In [4]:
# Designate g0 ===================================

def get_clusters(num_roots: int, max_hops: int, target_size: int) -> torch.Tensor:
  root_nodes = torch.rand(adj.shape[0]).topk(num_roots).indices

  for hop in range(max_hops):
    newNodes = adj[root_nodes].nonzero().t()[1]
    root_nodes = torch.cat((root_nodes, newNodes))
    root_nodes = torch.unique(root_nodes)
    if root_nodes.shape[0] >= target_size:
      break

  g0 = torch.zeros(adj.shape[0])
  g0[root_nodes[:target_size]] = 1
  g0 = g0.bool()
  return g0

g0 = torch.tensor([0])
if args.g0_method == 'many_clusters': # 10 nodes and their neighbors
  g0 = get_clusters(10, 10, g0_size)
elif args.g0_method == 'large_cluster': # 1 node and its neighbors
  g0 = get_clusters(1, 10, g0_size)
elif args.g0_method == 'random': # g0 is random/bias
  g0_probs = torch.ones(adj.shape[0])
  g0_probs = g0_probs * (g0_size / g0_probs.sum())
  g0_probs.clamp_(0, 1)
  g0 = torch.bernoulli(g0_probs).bool()
elif args.g0_method == 'bias': # g0 is skewed toward a class by factor of 3
  bias = torch.randint(0, int(labels.max()) + 1, [1]).item()
  print(f'G0 class bias: {bias}')
  g0_probs = torch.ones(adj.shape[0])
  g0_probs[labels == bias] = 3
  g0_probs = g0_probs * (g0_size / g0_probs.sum())
  g0_probs.clamp_(0, 1)
  g0 = torch.bernoulli(g0_probs).bool()

print(f'G0 size: {g0.sum().item()}')
print(f'G0 pct: {g0.sum().item() / adj.shape[0]:.2%}')

# g0 = g0.cpu()
gX = ~g0

G0 size: 519
G0 pct: 20.89%


# Method

In [5]:
import torch

def scale(M: torch.Tensor, epsilon: int, patience=5) -> torch.Tensor:
    if M.abs().sum() == 0: return M
    
    for i in range(patience): # Maximum attempts
        if abs(epsilon / M.abs().sum() - 1) < 0.1: return M.clamp(-1, 1) # Stop with early convergence
        M = (M * (epsilon / M.abs().sum())).clamp(-1, 1)

    return M.clamp(-1, 1)

def discretize(M: torch.Tensor) -> torch.Tensor:
    return torch.bernoulli(M.abs().clamp(0, 1))

def truncate(M: torch.Tensor, A: torch.Tensor) -> torch.Tensor:
    """
    Truncates values in M such that only:
    1. positive values exist in M corresponding to non-existing edges
    2. negative values exist in M corresponding to alread existing edges
    """
    assert M.shape == A.shape
    negative_vals = (M * A).clamp(max=0)
    positive_vals = (M * (1-A)).clamp(min=0)
    return positive_vals + negative_vals

def xor(A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:
    return (A + B) - torch.mul(A * B, 2)

In [84]:
def evaluate(X, A, T, g, train_mask, val_mask, device='cpu'):
    X = X.to(device)
    A = A.to(device)
    θ = GCN(nfeat=X.shape[1], nclass=T.max().item()+1, nhid=32, device=device).to(device)
    masked = train_mask.clone()
    masked[g] = 0
    θ.fit(X, A, T, masked, val_mask, train_iters=100)
    pred = θ(X, A).cpu()
    result = pred.argmax(1) == T
    acc_g0 = result[g].sum().item() / g.sum().item()
    acc_gX = result[~g].sum().item() / (~g).sum().item()
    print(f"G0: {acc_g0:.2%}")
    print(f"GX: {acc_gX:.2%}")
    return (acc_g0, acc_gX)

# Baselines

In [77]:
def CER(A: torch.Tensor, g0: torch.Tensor) -> torch.Tensor:
    A = A.cpu()
    g0 = g0.cpu()
    A[:, g0] = 0
    A[g0, :] = 0
    return A

def REF(A: torch.Tensor, g0: torch.Tensor, ε: int) -> torch.Tensor:
    noise = torch.zeros_like(A)
    noise[g0, :] = 1
    noise[:, gX] = 0
    noise *= 2 * ε / noise.sum()
    noise = torch.bernoulli(noise.clamp(0, 1))
    return xor(A, noise)

In [98]:
locked_adj = CER(adj, g0)
evaluate(feat, locked_adj, T_s, g0, train_mask, val_mask, device)
evaluate(feat, locked_adj, T_u, g0, train_mask, val_mask, device)

budget = (adj.shape[0]) * args.budget_pct
locked_adj = REF(adj, g0, budget)
evaluate(feat, locked_adj, T_s, g0, train_mask, val_mask, device)
evaluate(feat, locked_adj, T_u, g0, train_mask, val_mask, device)

G0: 12.72%
GX: 79.25%
G0: 100.00%
GX: 100.00%
G0: 18.69%
GX: 79.09%
G0: 100.00%
GX: 100.00%


(1.0, 1.0)

# SLL

In [73]:
from deeprobust.graph.defense import GCN
from tqdm import tqdm
import torch.nn.functional as F

def SLL(
        A: torch.Tensor, 
        X:torch.Tensor, 
        g0: torch.Tensor, 
        T_s: torch.Tensor, 
        T_u: torch.Tensor, 
        ε: int,
        epochs: int,
        lr: float,
        train_mask: torch.Tensor,
        val_mask: torch.Tensor,
        surrogate_lr=1e-2,
        device='cpu') -> torch.Tensor:
    """
    T_s is sensitive classification task
    T_u is utility classification task
    g0 is array of boolean representing protected nodes.
    """
    gX = ~g0
    X = X.to(device)
    T_s = T_s.to(device)
    T_u = T_u.to(device)
    A = A.to(device)

    M = torch.zeros_like(A).float().to(device)
    θ_s = GCN(nfeat=X.shape[1], nclass=T_s.max().item()+1, nhid=32, lr=surrogate_lr, device=device).to(device)
    θ_u = GCN(nfeat=X.shape[1], nclass=T_u.max().item()+1, nhid=32, lr=surrogate_lr, device=device).to(device)

    t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    A_p = torch.zeros_like(A).to(device).requires_grad_(True) # Initialize modified adj

    for epoch in t:
        # A_p = xor(A, discretize(M, ε)).to(device).requires_grad_(True) # To clear graident of modified adj
        L = 0
        sens_pred = θ_s(X, A_p)
        L += F.cross_entropy(sens_pred[g0], T_s[g0]) \
            - F.cross_entropy(sens_pred[gX], T_s[gX])
        
        utility_pred = θ_u(X, A_p)
        L -= F.cross_entropy(utility_pred, T_u)

        A_grad = torch.autograd.grad(L, A_p)[0]
        M = truncate(M + ((lr * A_grad) / (epoch + 1)), A)
        # M = scale(M, ε)
        M = utils.projection(M, ε)

        A_p = xor(A, discretize(M)).to(device).requires_grad_(True)
        θ_s.fit(X, A_p, T_s, train_mask, val_mask, train_iters=1)
        θ_u.fit(X, A_p, T_u, train_mask, val_mask, train_iters=1)

        t.set_postfix({
            "loss": L.item(),
            # "edges modified": (A_p.cpu() - A).abs().sum().item()
        })
    
    return A_p.requires_grad_(False)

budget = (adj.shape[0]) * args.budget_pct
locked_adj = SLL(adj, feat, g0, T_s, T_u, budget, 100, lr=10, train_mask=train_mask, val_mask=val_mask, device=device)
evaluate(feat, locked_adj, T_s, g0, train_mask, val_mask, device)
evaluate(feat, locked_adj, T_u, g0, train_mask, val_mask, device)

100%|██████████| 100/100 [00:05<00:00, 17.01it/s, loss=93.4]


G0: 37.96%
GX: 75.13%
G0: 99.42%
GX: 99.39%


# SLL Grad

In [10]:
class SamplingMatrix:
    def __init__(self, num_nodes: int, g0: torch.Tensor) -> None:
        self.g0_idx = g0.nonzero()
        self.gX_idx = (~g0).nonzero()
        self.n = num_nodes
        self.r00 = 1/3
        self.r0X = 1/3
        self.rXX = 1/3
        pass

    def update_ratio(self, A_grad, ema_k=3):
        sum_00 = A_grad[self.g0_idx, self.g0_idx].sum()
        sum_XX = A_grad[self.gX_idx, self.gX_idx].sum()
        sum_0X = A_grad.sum() - sum_00 - sum_XX
        total = sum_00 + sum_0X + sum_XX
        self.r00 = ((ema_k - 1) * self.r00 +  (sum_00 / total)) / ema_k
        self.r0X = ((ema_k - 1) * self.r0X +  (sum_0X / total)) / ema_k
        self.rXX = ((ema_k - 1) * self.rXX +  (sum_XX / total)) / ema_k
        total = self.r00 + self.r0X + self.rXX
        self.r00 = self.r00 / total
        self.r0X = self.r0X / total
        self.rXX = self.rXX / total

    def get_sample(self, sample_size: int) -> torch.Tensor:
        num_00 = int(self.r00 * sample_size)
        g00 = torch.cat(
            [self.g0_idx[torch.randint(0, self.g0_idx.shape[0], [num_00])], 
             self.g0_idx[torch.randint(0, self.g0_idx.shape[0], [num_00])]]
             , 1)
        num_0X = int(self.r0X * sample_size)
        g0X = torch.cat(
            [self.g0_idx[torch.randint(0, self.g0_idx.shape[0], [num_0X])], 
             self.gX_idx[torch.randint(0, self.gX_idx.shape[0], [num_0X])]]
             , 1)
        num_XX = int(self.rXX * sample_size)
        gXX = torch.cat(
            [self.gX_idx[torch.randint(0, self.gX_idx.shape[0], [num_XX])], 
             self.gX_idx[torch.randint(0, self.gX_idx.shape[0], [num_XX])]]
             , 1)
        
        return torch.cat([g00, g0X, gXX], 0).t()

sampling = SamplingMatrix(adj.shape[0], g0)
sample_size = int(adj.shape[0] * adj.shape[1] * 0.1)
sample = sampling.get_sample(sample_size)
adj[sample[0], sample[1]].sum(), sample_size

(tensor(996.), 617522)

In [11]:
def SLL_G(
        A: torch.Tensor, 
        X:torch.Tensor, 
        g0: torch.Tensor, 
        T_s: torch.Tensor, 
        T_u: torch.Tensor, 
        ε: int,
        epochs: int,
        sample_ct: int,
        sample_size: int,
        lr: float,
        train_mask: torch.Tensor,
        val_mask: torch.Tensor,
        device='cpu') -> torch.Tensor:
    """
    T_s is sensitive classification task
    T_u is utility classification task
    g0 is array of boolean representing protected nodes.
    """
    gX = ~g0
    X = X.to(device)
    T_s = T_s.to(device)
    T_u = T_u.to(device)
    A = A.to(device)

    M = torch.zeros_like(A).float()
    θ_s = GCN(nfeat=X.shape[1], nclass=T_s.max().item()+1, nhid=32, device=device).to(device)
    θ_u = GCN(nfeat=X.shape[1], nclass=T_u.max().item()+1, nhid=32, device=device).to(device)

    t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    A_p = torch.zeros_like(A) # Initialize modified adj
    sampling_matrix = SamplingMatrix(A.shape[0], g0)

    for epoch in t:
        # A_p = xor(A, discretize(M, ε)).to(device).requires_grad_(True) # To clear graident of modified adj
        cts = torch.zeros_like(A, dtype=torch.int)
        A_grad = torch.zeros_like(A, dtype=torch.float)
        c_L = 0

        for _ in range(sample_ct):
            idx = sampling_matrix.get_sample(sample_size)

            sample = A_p[idx[0], idx[1]].clone().detach().requires_grad_(True).to(device)
            A_p[idx[0], idx[1]] = sample

            L = 0
            sens_pred = θ_s(X, A_p)
            utility_pred = θ_u(X, A_p)
            L += F.cross_entropy(sens_pred[g0], T_s[g0]) \
                - F.cross_entropy(sens_pred[gX], T_s[gX])
            L -= F.cross_entropy(utility_pred, T_u)

            grad = torch.autograd.grad(L, sample)[0]
            cts[idx[0], idx[1]] += 1
            A_grad[idx[0], idx[1]] += grad
            c_L += L.item()

        A_grad = torch.div(A_grad, cts)
        A_grad[A_grad != A_grad] = 0

        sampling_matrix.update_ratio(A_grad)
        M = truncate(M + ((lr * A_grad) / (epoch + 1)), A)
        # M = scale(M, ε)
        M = utils.projection(M, ε)

        A_p = xor(A, discretize(M)).to(device)
        θ_s.fit(X, A_p, T_s, train_mask, val_mask, train_iters=1)
        θ_u.fit(X, A_p, T_u, train_mask, val_mask, train_iters=1)

        t.set_postfix({
            "loss": c_L,
            # "edges modified": (A_p.cpu() - A).abs().sum().item()
        })
    
    return A_p.requires_grad_(False)


In [12]:
def SLL_G2(
        A: torch.Tensor, 
        X:torch.Tensor, 
        g0: torch.Tensor, 
        T_s: torch.Tensor, 
        T_u: torch.Tensor, 
        ε: int,
        epochs: int,
        sample_ct: int,
        sample_size: int,
        lr: float,
        device='cpu') -> torch.Tensor:
    """
    T_s is sensitive classification task
    T_u is utility classification task
    g0 is array of boolean representing protected nodes.
    """
    gX = ~g0
    X = X.to(device)
    T_s = T_s.to(device)
    T_u = T_u.to(device)
    A = A.to(device)

    M = torch.zeros_like(A).float()
    θ_s = GCN(nfeat=X.shape[1], nclass=T_s.max().item()+1, nhid=32, device=device).to(device)
    θ_u = GCN(nfeat=X.shape[1], nclass=T_u.max().item()+1, nhid=32, device=device).to(device)

    t = tqdm(range(epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
    A_p = torch.zeros_like(A).to(device).requires_grad_(True) # Initialize modified adj
    sampling_matrix = SamplingMatrix(A.shape[0], g0)

    for epoch in t:
        # A_p = xor(A, discretize(M, ε)).to(device).requires_grad_(True) # To clear graident of modified adj
        # cts = torch.zeros_like(A, dtype=torch.int)

        L = 0
        sens_pred = θ_s(X, A_p)
        L += F.cross_entropy(sens_pred[g0], T_s[g0]) \
            - F.cross_entropy(sens_pred[gX], T_s[gX])
        
        utility_pred = θ_u(X, A_p)
        L -= F.cross_entropy(utility_pred, T_u)

        temp_grad = torch.autograd.grad(L, A_p)[0]
        idx = sampling_matrix.get_sample(sample_size * num_samples)
        # cts[idx[0], idx[1]] += 1
        A_grad = torch.zeros_like(A, dtype=torch.float)
        A_grad[idx[0], idx[1]] = temp_grad[idx[0], idx[1]]
        # A_grad = torch.div(A_grad, cts)
        # A_grad[A_grad != A_grad] = 0
        sampling_matrix.update_ratio(A_grad)

        M = truncate(M + ((lr * A_grad) / (epoch + 1)), A)
        # M = scale(M, ε)
        M = utils.projection(M, ε)

        A_p = xor(A, discretize(M)).to(device).requires_grad_(True)
        θ_s.fit(X, A_p, T_s, train_mask, idx_val, train_iters=1)
        θ_u.fit(X, A_p, T_u, train_mask, idx_val, train_iters=1)

        t.set_postfix({
            "loss": L.item(),
            # "edges modified": (A_p.cpu() - A).abs().sum().item()
        })
    
    return A_p.requires_grad_(False)

# Eval

In [70]:
budget = (adj.shape[0]) * args.budget_pct
sample_size = int(adj.shape[0] * adj.shape[1] * 0.001)
locked_adj = SLL_G(adj, feat, g0, T_s, T_u, budget, sample_ct=10, sample_size=sample_size, epochs=30, lr=100, train_mask=train_mask, val_mask=val_mask, device=device)
evaluate(feat, locked_adj, T_s, g0, train_mask, val_mask, device)
evaluate(feat, locked_adj, T_u, g0, train_mask, val_mask, device)

100%|██████████| 30/30 [00:08<00:00,  3.66it/s, loss=113] 


G0: 50.29%
GX: 81.64%
G0: 99.42%
GX: 99.39%
